In [13]:
import geopandas as gpd
import vaex
import glob
import os

In [23]:
df_iptu = vaex.open('dados/IPTU_2022.hdf5')

In [30]:
# df_iptu['TIPO DE USO DO IMOVEL'].value_counts()

## SELECIONAR SEGUNDO OS CRITERIOS
# 'Residência' ou 'Residência coletiva, exclusive cortiço (mais de uma residência no lote)' ou 'Residência e outro uso (predominância residencial)'

## Existem alguns parametros interessantes para qualificá-las
# 'AREA DO TERRENO',
#  'AREA CONSTRUIDA',
#  'AREA OCUPADA',
#  'VALOR DO M2 DO TERRENO',
#  'VALOR DO M2 DE CONSTRUCAO',
#  'ANO DA CONSTRUCAO CORRIGIDO',
#  'QUANTIDADE DE PAVIMENTOS',
#  'TESTADA PARA CALCULO',
#  'TIPO DE PADRAO DA CONSTRUCAO',
# 'QUANTIDADE DE ESQUINAS FRENTES',

# Além dos dados LiDAR de Vegetação e Morfologia

## Que tal

In [32]:
# df_iptu.column_names

In [22]:
for f in glob.glob('dados/lotes/*'):
    file = f'zip://{f}!{os.path.split(os.path.splitext(f)[0])[-1]}'
    gdf_lotes = gpd.read_file(file)
    break

In [8]:
df_iptu

#,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,NUMERO DO CONDOMINIO,CODLOG DO IMOVEL,NOME DE LOGRADOURO DO IMOVEL,NUMERO DO IMOVEL,COMPLEMENTO DO IMOVEL,BAIRRO DO IMOVEL,REFERENCIA DO IMOVEL,CEP DO IMOVEL,QUANTIDADE DE ESQUINAS FRENTES,FRACAO IDEAL,AREA DO TERRENO,AREA CONSTRUIDA,AREA OCUPADA,VALOR DO M2 DO TERRENO,VALOR DO M2 DE CONSTRUCAO,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
0,0010030001-4,2022,1,2022-01-01 00:00:00.000000000,00-0,03812-1,R S CAETANO,13.0,--,SANTA EFIGENIA,--,01104-001,1,1.0,136,135,108,3340.0,2465.0,1924,1,13.0,Loja,Comercial horizontal - padrão B,De esquina,0.2,1963,1,0
1,0010030002-2,2022,1,2022-01-01 00:00:00.000000000,00-0,03812-1,R S CAETANO,19.0,--,SANTA EFIGENIA,--,01104-001,0,1.0,90,67,67,3340.0,2465.0,1944,1,6.0,Loja,Comercial horizontal - padrão B,Normal,0.2,1963,1,0
2,0010030003-0,2022,1,2022-01-01 00:00:00.000000000,00-0,03812-1,R S CAETANO,27.0,--,SANTA EFIGENIA,--,01104-001,0,1.0,105,140,84,3340.0,2465.0,1965,2,7.85,Loja,Comercial horizontal - padrão B,Normal,0.26,1963,1,0
3,0010030004-9,2022,1,2022-01-01 00:00:00.000000000,00-0,03812-1,R S CAETANO,33.0,--,--,--,01104-001,0,1.0,108,103,86,3340.0,2465.0,1944,1,6.05,Loja,Comercial horizontal - padrão B,Normal,0.2,1963,1,0
4,0010030005-7,2022,1,2022-01-01 00:00:00.000000000,00-0,03812-1,R S CAETANO,39.0,--,--,--,01104-001,0,1.0,120,170,110,3340.0,2465.0,2004,2,6.7,Loja,Comercial horizontal - padrão B,Normal,0.84,1963,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"3,580,056",3101180238-1,2022,1,2022-01-01 00:00:00.000000000,01-9,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 42,JABAQUARA,BL 15 QD II,04309-011,5,0.0033,18041,84,5312,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0
"3,580,057",3101180239-1,2022,1,2022-01-01 00:00:00.000000000,01-9,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 43,JABAQUARA,BL 15 QD II,04309-011,5,0.0033,18041,84,5312,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0
"3,580,058",3101180240-3,2022,1,2022-01-01 00:00:00.000000000,01-9,02235-7,AV ENG ARMANDO DE ARRUDA PEREIRA,1827.0,AP 44,JABAQUARA,BL 15 QD II,04309-011,5,0.0033,18041,84,5312,2301.0,2005.0,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,De esquina,0.2,1985,1,0
"3,580,059",3101190001-4,2022,1,2022-01-01 00:00:00.000000000,00-0,19137-0,R COTIPORA,nan,--,CIDADE VARGAS,--,04317-060,2,1.0,18500,0,0,2149.0,0.0,0,0,0.0,Terreno,TERRENO,De esquina,0.0,1988,1,0
